<a href="https://colab.research.google.com/github/ava-28/Transformer/blob/main/transformer01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GPUtil

In [ ]:
!git clone https://github.com/ava-28/TransformerNo1.git


Cloning into 'TransformerNo1'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!git clone https://github.com/YOUR_USERNAME/YOUR_REPO.git



Cloning into 'YOUR_REPO'...
fatal: could not read Username for 'https://github.com': No such device or address


### Scaled Dot-Product Attention

This is the core of the attention mechanism. It takes three inputs: Query (Q), Key (K), and Value (V). It computes the dot product of the query with all keys, divides by the square root of the dimension of the keys ($√ d_k$), and then applies a softmax function to get the attention weights. These weights are then multiplied by the values to get the output.

The formula is:
$$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $$

In [ ]:
import warnings
warnings.filterwarnings("ignore")
!pip install -q torchdata==0.7.1 torchtext==0.18.0 spacy==3.8.0 altair GPUtil
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 53.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:


import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

OSError: Could not load this library: /usr/local/lib/python3.12/dist-packages/torchtext/lib/libtorchtext.so

In [ ]:
# Some convenience helper functions used throughout the notebook


def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None


In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

In [ ]:
def clones(module, N):
  "produce N identical layers"
  return nn.ModuleList([copy.deepcopy(module) for _ in range (N)])

In [ ]:
from torch.nn.modules.normalization import LayerNorm
class Encoder(nn.Module):
  "Core ecnoder is a stack of N layers"
  def __init__(self, layer, N):
    super(Encoder,self).__init__()
    self.layers = clones(layer, N)
    self.norm = LayerNorm(layer.size)

def forward(self, x, mask):
  "pass the input (and mask) throuhg each layer in turn"
  for layer in self.layers:
    x = layer(x, mask)
  return self.norm(x)

In [ ]:
"defining the LayerNorm class"
class LayerNorm(nn.Module):
  "Construct a layernorm module"
  def __init__ (self, features, eps = 1e-6):
    super(LayerNorm, self). __init__()
    self.a_2 = nn.Parameter(torch.ones(features))
    self.b_2 = nn.Parameter(torch.zeros(features))
    self.eps = eps

  def forward(self, x):
    mean = x.mean(-1, keepim =True)
    std = x.std(-1, keepdim = True )
    return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
class SublayerConnection (nn.Module):
  """
  A residual connertion followed by a layer norm.
  """
  def __init__ (self,size, dropout):
    super(SublayerConnection, self).__init__()
    self.norm = LayerNorm(size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, sublayer):
    "Apply residual connection to any sublayer with the same size"
    return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderLayer (nn.Module):
  "Encoder is made up from self-attention and feed forward"
  def __init__(self, size, self_attn, feed_forward, dropout):
    super(EncoderLayer, self).__init__()
    self.self_attn = self_attn
    seld.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size,dropout), 2)
    self.size = size




  def forward(self, x, mask):
    #So:  self.sublayer[0] → Add & Norm wrapper for self-attention.   self.sublayer[1] → Add & Norm wrapper for FFN#
    #self.self_attn(x, x, x, mask) means: Query = Key = Value = x.      ---> Tokens attend to each other#

    x = self.sublayer[0](
        x,
        lambda x: self,self_attn(x,x,x, mask)
    )
    return self.sublayer [1] (x, self.feed_forward)

Here we start with the Decoder:


the decoder is cpomposed of a stack pf N = 6 identical layers. So we use the same N here as well.





In [ ]:
class Decoder(nn.Module):
  "Generic N layer decoder with masking"

    def __init__(self, layer, N):
      super(Decoder, self).__init__()
      self.layers = clones(layer, N)
      self.norm = LayerNorm (layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
      for layer in self.layers:
        x = layer (x, memory, src_mask, tgt_mask)
      return self.norm

we have three parts here:
1. self attention (over the output of the the decoder)
2. sequence attention (over the output of both encoder and the decoder0
3. Feed forward

we have to also employ the residual connections around each of the sub-layers of the decoder, followed by layer normalization.

In [ ]:
class DecoderLayer(nn.Module):
  "Decoder is made of self-attn, and feed froward. we have the size for it, the initial input of the decoder via self attention(what it had produced before)"
  "the multi haed attention for the encoder decoder layer, the feed forwrd, and the dropout"

  def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
    super(DecoderLayer, self).__inti__()
    self.size = size
    self.self_attn = self_attn
    self.src_attn = self_attn
    self.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size, dropout), 3)


    def forward(self, x, memory, src_mask, tgt_mask):

      m = memory
      x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
      x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
      return slef.sublayer[2](x, self.feed_forward)

In [ ]:
def subsequent_mask (size):
  "Mask out subsequent positions."
  attn_shape = (1, size, size)
  subsequence_mask = troch.triu(torch.ones(attn_shape), diagonal=1).type(
      torch.uint8
      # torch.uint8 = Casts the tensor to an older boolean-ish integer type (0/1)#
  )
  return subsequent_mask == 0